In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,GlobalAveragePooling2D

# Image preparation

In [ ]:
train_dir = "./datasets/train"
validation_dir = "./datasets/validation"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator(rescale = 1.0/255.0)
validation_datagen  = ImageDataGenerator(rescale = 1.0/255.0)

# --------------------
# Flow training images in batches of 32 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    target_size=(256, 256))     

# --------------------
# Flow validation images in batches of 16 using validation_datagen generator
# --------------------
validation_generator =  validation_datagen.flow_from_directory(validation_dir,
                                                                 batch_size=16,
                                                                 class_mode  = 'binary',
                                                                 target_size = (256, 256))


# Baseline Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (7,7), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # fully connected layer with 256 hidden units
    tf.keras.layers.Dense(256, activation='relu'),
    # final sigmoid layer for classification
    tf.keras.layers.Dense(1, activation='sigmoid')  
])
model.summary()

# Improved Model with inception v3

In [ ]:
# download the weight for inceptionv3
# https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
# put it in weight folder

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = './weight/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (256, 256, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed4')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 256 hidden units and ReLU activation
x = layers.Dense(256, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model(pre_trained_model.input, x) 

In [ ]:
model.summary()

# Improved Model with Xception

In [ ]:
from tensorflow.keras.applications.xception import Xception

pre_trained_model = Xception(weights='imagenet', 
                            include_top=False, 
                            input_shape=(256, 256, 3))

In [ ]:
pre_trained_model.summary()

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('add_8')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 256 hidden units and ReLU activation
x = layers.Dense(256, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)           

model = Model(pre_trained_model.input, x) 

In [ ]:
model.summary()

# Improved model with VGG16

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator(rescale = 1.0/255.0)
validation_datagen  = ImageDataGenerator(rescale = 1.0/255.0)

# --------------------
# Flow training images in batches of 32 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    target_size=(224, 224))     

# --------------------
# Flow validation images in batches of 16 using validation_datagen generator
# --------------------
validation_generator =  validation_datagen.flow_from_directory(validation_dir,
                                                                 batch_size=16,
                                                                 class_mode  = 'binary',
                                                                 target_size = (224, 224))


In [ ]:
# download weight from
# https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
# put it in weight folder

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

local_weights_file = './weight/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = VGG16(input_shape = (224, 224, 3), 
                            include_top = False, 
                            weights = None) # "imagenet"

pre_trained_model.load_weights(local_weights_file)

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
model = Sequential()
model.add(pre_trained_model)
model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

# Improved Model with MobileNet

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#rescale all image by 1/255
train_datagen = ImageDataGenerator(rescale = 1.0/255.)
validation_datagen = ImageDataGenerator(rescale = 1.0/255.)

# --------------------
# Flow training images in batches of 32 using train_datagen generator (supaya tidak overfitting pada training)
# --------------------
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   batch_size = 32,
                                                   class_mode = 'binary',
                                                   target_size = (224, 224))
# --------------------
# Flow validation images in batches of 16 using validation_datagen generator
# --------------------
validation_generator = train_datagen.flow_from_directory(validation_data_dir,
                                                   batch_size = 16,
                                                   class_mode = 'binary',
                                                   target_size = (224, 224))

In [ ]:
base_model = MobileNet(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

base_model.summary()

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(224, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

# Compile and Fit

In [ ]:
from tensorflow.keras.optimizers import RMSprop
# RMSprop(learning_rate=0.001), adam

model.compile(optimizer=RMSprop(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.fit(train_generator,
            validation_data=validation_generator,
            steps_per_epoch=25,
            epochs=10,
            validation_steps=12)

# Visualization of the accuracy and loss

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = model.history.history['accuracy']
val_acc  = model.history.history['val_accuracy']
loss     = model.history.history['loss']
val_loss = model.history.history['val_loss']

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.legend(["train", "validation"])
plt.title('Training and validation accuracy')
plt.show()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.legend(["train", "validation"])
plt.title('Training and validation loss')
plt.show()

# Saving Model

In [ ]:
model.save("./model/cataract_model.h5")

In [ ]:
# convert h5 to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('./model/cataract_model.tflite', 'wb') as f:
    f.write(tflite_model)